In [2]:
import pandas as pd
import numpy as np
import unicodedata
import statistics
import operator

from fuzzywuzzy import fuzz 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from sklearn.decomposition import NMF

from SToC import *

import os

from IPython.display import HTML, display
from tabulate import tabulate
from metrics import Evaluation as Eval

from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

/home/leandror/.local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [60]:
with open('document_dict.pickle', 'rb') as f:
    document_dict= pickle.load(f)

In [61]:
document_dict

{'hardware': [32,
  47,
  56,
  77,
  105,
  107,
  128,
  129,
  139,
  146,
  177,
  287,
  302,
  317,
  344,
  351,
  352,
  357,
  381,
  386,
  388,
  393,
  412,
  462,
  469,
  471,
  481,
  497,
  503,
  505,
  510,
  530,
  610,
  657,
  661,
  682,
  688,
  698,
  727,
  754,
  798,
  801,
  836,
  863,
  877,
  906,
  913,
  918,
  922,
  933,
  968,
  972,
  986,
  989,
  1011,
  1040,
  1055,
  1057,
  1062,
  1070,
  1107,
  1144,
  1148,
  1210,
  1225,
  1229,
  1230,
  1233,
  1263,
  1342,
  1346,
  1352,
  1358,
  1382,
  1410,
  1434,
  1435,
  1453,
  1490,
  1499,
  1514,
  1537,
  1538,
  1556,
  1591,
  1613,
  1614,
  1622,
  1631,
  1634,
  1641,
  1716,
  1727,
  1729,
  1734,
  1763,
  1765,
  1795,
  1898,
  1901,
  1918,
  1919,
  1920,
  1927,
  2021,
  2028,
  2054,
  2067,
  2124,
  2151,
  2182,
  2186,
  2187,
  2193,
  2194,
  2264,
  2269,
  2276,
  2291,
  2296,
  2306,
  2341,
  2387,
  2392],
 'software and its engineering': [23,
  48,
  55,
  5

In [29]:
df_artigos = pd.read_csv('../../datasets/artigos/preprocessed/without_spacy/artigos2017_pre',sep="|",index_col=0)

In [32]:
df_artigos['titulo']

1       factor associated weight change weight managem...
2       regression base performance modeling provision...
3              model estimate change propagation software
5       survey comparative study time dynamic resource...
6               criterion resource allocation time system
7              task allocation time video stream decoding
8         hardware accelerate analysis time networks chip
9       model swarm robotic surveillance base automato...
10        automaton ant memory model foraging swarm robot
31                             optimization base analysis
32      artificial neural network classification analy...
33                             pigment bacteria associate
35                        derive experiment line approach
36                           detect inconsistency dataset
37      manage provenance implicit data flows scientif...
38               query provenance along domain data using
39      tool collecting analyzing manage provenance py...
41      map de

In [16]:
document_dict['software and its engineering']

144

In [5]:
def execute_tfidf(dataset,max_df=1,min_df=1,ngram=(0,2),stop_words=[]):
    tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,ngram_range=ngram,stop_words=stop_words)
    tfidf = tfidf_vectorizer.fit_transform(dataset)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return tfidf,tfidf_feature_names
def execute_nmf(tfidf,n_topics = 10,n_components = 5):
    n_topics = n_topics
    n_components = n_components
    nmf = NMF(n_components=n_components,max_iter=400)
    W = nmf.fit_transform(tfidf)
    H = nmf.components_.transpose() 
    return nmf,W,H
def print_top_words_display(H, feature_names, n_top_words):
    table_output = [['Topic','topwords']]
    topics = []
    for topic_idx, topic in enumerate(H):
        top_index = ["Topic # %d" % topic_idx]
        top_words = [" ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])]
        values = [value for value in np.sort(topic)[:-n_top_words - 1:-1]]
        table_output.append(top_index + top_words)
        topics.append(" ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    df_out = pd.DataFrame(table_output, index=None)
    display(HTML(tabulate(table_output, tablefmt='html')))
    return topics,df_out

In [63]:
f = open('../acm_words.txt','r')
acm_words = f.read().replace('\n',' ').split()
for keys in document_dict.keys():
    new_df = df_artigos.iloc[document_dict[keys]]
    artigos = new_df['titulo'].values
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(artigos)
    sum_words = X.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    #print(words_freq)
    indices = np.arange(len(words_freq))
    word = []
    frequency = []
    for i in range(len(words_freq)):
        word.append(words_freq[i][0])
        frequency.append(words_freq[i][1])
    stop_words_without_acm_words = list(set(word) - set(acm_words))
    # TF-IDF
    max_df = 0.95
    min_df = 0.01
    ngram= (1,1)
    #NMF
    n_components = 8
    n_topics = 10
    # STOC
    #n_final = 30
    tfidf,tfidf_feature_names = execute_tfidf(artigos,max_df=max_df,min_df=min_df,ngram=ngram,stop_words=stop_words_without_acm_words)
    nmf,W,H = execute_nmf(n_components=n_components,n_topics=n_topics,tfidf=tfidf)
    topics_nmf,a = print_top_words_display(H.transpose(), tfidf_feature_names,n_topics)
    np.savez('out/topics_'+keys+'.npz',H = H.transpose(),tfidf_feature_names = tfidf_feature_names,n_topics=n_topics)


Topic,topwords
Topic # 0,heuristic base algorithm path domain identification evolutionary allocation data class
Topic # 1,level cloud base software modeling model application caching set context
Topic # 2,service time cloud software application science architectures management evaluation data
Topic # 3,research education design action analysis testing science use wireless context
Topic # 4,solution exact system time use analysis application model manufacturing information
Topic # 5,networks neural tree base use wireless architectures problems designing model
Topic # 6,product line configuration class software base diagrams application system environment
Topic # 7,network data management power allocation generation infrastructure high wireless development


Topic,topwords
Topic # 0,heuristic search solution algorithm path bound domain base data allocation
Topic # 1,engineering software analysis computer education process network system learning mapping
Topic # 2,generation use network column networks design power energy mesh allocation
Topic # 3,multi systems agent optimization management architecture monitoring integration tree framework
Topic # 4,environment development self interference checking factor authentication code analysis program
Topic # 5,location facility allocation constraint geometric tree algorithm cluster problems model
Topic # 6,algorithms evaluation problems analysis optimization performance graphs exact genetic application
Topic # 7,base model system algorithm interoperability mesh code approaches classification test


Topic,topwords
Topic # 0,base event matching type web cluster authentication similarity social user
Topic # 1,research design management context networks action synthesis non platform testing
Topic # 2,activity evolution structure physical synthesis neural area co model time
Topic # 3,computer application performance programming simulator vision detection interface architecture learning
Topic # 4,literature computing health cloud mobile software system self web process
Topic # 5,education distance higher learning game student use development knowledge children
Topic # 6,engineering software process system model business mapping systems context development
Topic # 7,analysis use science network program mining management technology environment learning


Topic,topwords
Topic # 0,analysis image type reduction processing programs complexity platform complex decision
Topic # 1,use image recognition prediction classification architectures literature failure system programming
Topic # 2,service cloud time level software solution perception data architectures management
Topic # 3,mining data classification text model mapping process selection techniques development
Topic # 4,education learning distance research higher children complexity engineering student impact
Topic # 5,algorithms optimization problems evaluation design system max image graphs data
Topic # 6,prediction machine base model feature energy structure value time programming
Topic # 7,network management base infrastructure service knowledge field criterion allocation implementation


Topic,topwords
Topic # 0,analysis type complexity platform shape environment linear value similarity tools
Topic # 1,base matching modeling type knowledge similarity model clustering social student
Topic # 2,game design digital use evaluation student complexity quality agent platform
Topic # 3,neural networks use network artificial recognition classification training social performance
Topic # 4,literature software computing mobile design system use quality process research
Topic # 5,model programming agent distance linear value student evaluation reliability tools
Topic # 6,education use research development distance engineering student mining knowledge learning
Topic # 7,mapping mobile environment approaches engineering tools software testing model matching


Topic,topwords
Topic # 0,search algorithm use software base parameter performance optimization theory non
Topic # 1,service cloud time level software data solution management mining architectures
Topic # 2,activity structure evolution research physical recognition time neural synthesis co
Topic # 3,location facility algorithm allocation constraint non games time problems tree
Topic # 4,networks base wireless sensor analysis optical detection implementation performance self
Topic # 5,feature base analysis space theory system recommendation shape games based
Topic # 6,network management research allocation design service infrastructure data neural implementation
Topic # 7,model use cluster allocation business impact social sensor wireless system


Topic,topwords
Topic # 0,product software line application configuration platform identification class evolution architecture
Topic # 1,solution service time exact manufacturing heuristic information analysis system clustering
Topic # 2,mining data classification text analysis algorithms service education development process
Topic # 3,generation column energy design network mesh networks power mobile evaluation
Topic # 4,mapping software energy process object engineering mobile interaction surface image
Topic # 5,computer engineering analysis application science vision programming simulator performance detection
Topic # 6,base model system mesh testing impact product models information manufacturing
Topic # 7,use diversity maps literature business allocation scheduling failure science management


Topic,topwords
Topic # 0,product software line base application system mapping testing configuration architecture
Topic # 1,research analysis education design activity management science action engineering testing
Topic # 2,service network management data solution time allocation science infrastructure level
Topic # 3,literature game quality software process mobile data base health analysis
Topic # 4,technology information implementation robotics management voltage project impact event single
Topic # 5,energy model performance networks distribution smart scenario context knowledge engineering
Topic # 6,cloud computing level system class time service performance analysis configuration
Topic # 7,use health mining context information design resource architectures communication identification


In [33]:
new_df = df_artigos.iloc[document_dict['hardware']]

In [36]:
new_df['titulo'].values

array(['system level design space identification core vision processor',
       'information technology service management processes small medium enterprises proposal implementation sequence',
       'heuristic tool detect move method opportunity',
       'cryptography wireless multimedia sensor networks survey research directions',
       'new strategies initialization training radial basis function neural networks',
       'heuristic find solution program',
       'multi objective evolutionary hyper heuristic applied integration test order problem',
       'hyper heuristic based product selection software product line testing',
       'data plane programmability beyond opportunity challenge network service operations management',
       'design science research methodology methodological strategy technological research',
       'update taxonomy network service management research field',
       'implementation deployment intelligent industrial wireless system induction motor monitori

Topic,topwords
Topic # 0,design noise rank range algorithms channel numerical optimization education base
Topic # 1,engineering critical evolution simulation process reality ontology science system based
Topic # 2,computing human social middle performance quantum solution simulation framework applied
Topic # 3,multi optimization integration energy architecture requirements layer location algorithms agent
Topic # 4,education code student distance quality children mining game methods line
Topic # 5,computer performance algorithm simulator care vision series detection interface program
Topic # 6,analysis energy requirements interface literature integration quality environment science use
Topic # 7,base line performance monitoring search layer time student management ontology
Topic # 8,interface methods computer game analysis systems application software vision evaluation
Topic # 9,network location energy virtual layer distance algorithms knowledge application social


In [14]:
def area_parents(area,areas_id,areas_parents,similar_area):
    li_id = getKeysByValue(areas_id,area)
    parents = [areas_parents[i] for i in li_id]
    for li in li_id:
        if li not in similar_area.keys():
            similar_area[li] = 1
        else:
            similar_area[li] += 1
    for parent in parents:
        for p in parent:
            if p not in similar_area.keys():
                similar_area[p] =1
            else:
                similar_area[p] +=1
                
        #print(area+"-->"+"-->".join([areas_id[i] for i in parent][::-1]))
    return similar_area
def topic_acm_area(topic,areas_id,areas_parents):
    similar_area = {}
    print(topic)
    m = 0
    for area in [areas_id[j] for j in y]:
        a = fuzz.partial_token_set_ratio(area,topic)
        if a > m:
            m = a
            s = area
        
             #area_parents(area,areas_id,areas_parents,similar_area)
    print(s)
    return similar_area
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

In [16]:
with open('../../datasets/'+ 'acm_hierachy' + '.pkl', 'rb') as f:
    acm = pickle.load(f)
with open('../../datasets/'+ 'acm_areas' + '.pkl', 'rb') as f:
    areas = pickle.load(f)
with open('../../datasets/'+ 'acm_areas_id' + '.pkl', 'rb') as f:
    areas_id = pickle.load(f)
with open('../../datasets/'+ 'acm_areas_parents' + '.pkl', 'rb') as f:
    areas_parents = pickle.load(f)

In [18]:
idx = getKeysByValue(areas_id,'software and its engineering')
y = []
for i in areas_parents:
    value = areas_parents[i]
    if idx[0] in value:
        if len(value) == 1: # Hieirarquias
            y.append(i)
[areas_id[j] for j in y]

['software organization and properties',
 'software notations and tools',
 'software creation and management']

In [23]:
table = []
for idx,topic in enumerate(topics_nmf):
    similar_area = topic_acm_area(topic,areas_id,areas_parents)
    print("---------------------------------------")
    print("\n")
    if similar_area.items():
        maior = 0
        for value in similar_area:
#             if not len(areas_parents[value]) == 0:
            if similar_area[value] > maior:
                maior = similar_area[value]
                print("a")
        for i in getKeysByValue(similar_area,maior):
            if len(areas_parents[i]) == 0:
                result = areas_id[i]
            
        print(result)
        table.append([idx,topic,result])

base detection reliability approaches student distance web video user development
software creation and management
---------------------------------------


game detection methods learning design time support algorithms engineering digital
software notations and tools
---------------------------------------


networks neural performance age fuzzy agent training classification activity estimation
software creation and management
---------------------------------------


quality context application network system networks impact life framework assessment
software creation and management
---------------------------------------


literature creation applications software network application detection social engineering design
software organization and properties
---------------------------------------


model age agent testing distance evaluation programming analysis student base
software organization and properties
---------------------------------------


education learning knowledge cre